In [ ]:
! pip install pillow
! pip install bs4

In [1]:
import requests
from bs4 import BeautifulSoup
from tkinter import *
from tkinter.ttk import Combobox
from PIL import ImageTk
import re

In [2]:
# GUI
# Webscrapping from cricX to get all the live score.
# Display the live match updates on the python app.

In [3]:
class CricketScore:

    def __init__(self, rootWindow):
        self.rootWindow = rootWindow
        self.rootWindow.title("LIVE CRICKET SCORE")
        self.rootWindow.geometry('1486x786')
        self.bg = ImageTk.PhotoImage(file='img.jpg')
        bg = Label(self.rootWindow, image=self.bg).place(x=0, y=0)

        # Let's make a label to the GUI
        self.label = Label(self.rootWindow, text='Live Scores', font=("times new roman", 60), compound='center').pack(padx=100, pady=50)
        # Fetch the live match details and set it in mh GUI
        self.var = StringVar()
        self.matches = self.match_details()
        self.data = [el for el in self.matches.keys()]
        self.cb = Combobox(self.rootWindow, values=self.data, width=50)
        self.cb.place(x=600, y=200, height=40)

        # Add a button to the GUI to check the details of the match
        self.btn = Button(self.rootWindow, text="check Score", font=("the new roman",15), command=self.show_match_details).place(x=700, y=250)



    # Function to hit the url to start scraping process.
    def scrap(self):
        url = 'https://crex.live/'
        page = requests.get(url)

        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find(id='slide')
        scrap_results = results.findAll(class_='live-card-middle')
        # print(scrap_results)
        return scrap_results
    



    # We will load all the matches details
    def match_details(self):

        # to scrap the data
        details = self.scrap()

        # Need to parse the and read the fetched data
        live_match = {}
        for detail in details:
            live_team_details = {}
            summary = self.match_summary(detail)
            if summary is not None:
                match_header = self.match_header(detail)
                teams = self.playing_teams(detail)
                scoreCard = self.scoreCards(detail)
                live_team_details['summary'] = summary
                live_team_details['match-header'] = match_header
                live_team_details['score'] = scoreCard[0] + " :: " + scoreCard[1]

                live_match[teams[0] + 'vs' + teams[1]] = live_team_details

        return live_match



    # Function to find the summary of single given match
    def match_summary(self, detail):
        finished_summary = detail.find('span', {'style':'color:var(--ce_highlight_ac3);'})
        toss_summary = detail.find('span', class_="comment")
        if finished_summary:
            return (finished_summary.text)
        elif toss_summary:
            return (toss_summary.text)
        

    # Function to fetch the match header of single given match
    def match_header(self, detail):
        header = detail.find('h3', class_="match-number").text
        return (header)
    
    # Function to fetch the playing team.
    def playing_teams(self, detail):
        t=[]
        teams = detail.findAll('div', class_='team-score')
        for team in teams:
            team_data = team.text
            index = re.search(r"\d", team_data).start() if re.search(r"\d", team_data) else len(team_data)
            t.append(team_data[:index])
        
        return t
    

    # Function to fetch the score-card of playing team.
    def scoreCards(self, detail):
        t=[]
        teams = detail.findAll('div', class_='team-score')
        for team in teams:
            team_data = team.text
            t.append(team_data)
        
        return t
        


    # Function to get the selected match
    def select_match(self):
        return self.cb.get()
    

    # Function to show scorecard on the GUI
    def show_match_details(self):

        # create a GUI frame to show the scorecard.
        self.frame = Frame(self.rootWindow, bg="#ADD8E6")
        self.frame.place(x=300, y=330, width=850, height=300)
        
        match_data = self.matches[self.select_match()]

        # display the team names
        Label(self.frame, text=self.select_match() + " - " + match_data['match-header'], font=("times new roman", 20, "bold"), bg="#ADD8E6", fg="red", bd=0).place(x=60, y=15)
        
        # display live score details.
        Label(self.frame, text="Live Score " + ":", font=("times new roman", 17, "bold"), bg="#ADD8E6", fg="black", bd=0).place(x=10, y=60)
       
        Label(self.frame, text= match_data['score'], font=("times new roman", 15, "bold"), bg="#ADD8E6", fg="black", bd=0).place(x=20, y=90)
        
        Label(self.frame, text="Summary :", font=("times new roman", 17, "bold"), bg="#ADD8E6", fg="black", bd=0).place(x=10, y=150)

        Label(self.frame, text= match_data['summary'], font=("times new roman", 15, "bold"), bg="#ADD8E6", fg="black", bd=0).place(x=20, y=180)

In [4]:
# main function to start to the application 
def main():

    # start the GUI window
    rootWindow = Tk()

    # Fetch the cricket score
    obj = CricketScore(rootWindow)

    # Keep on displaying the live score by updating the GUI
    rootWindow.mainloop()


try:
    # Execute the main file
    if __name__=='__main__':
        main()

except Exception as e:
    print(e)